<a href="https://colab.research.google.com/github/NithderP/NithderP/blob/main/Scalable_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 5: Initializing and downloading PySpark**
## Introduction to rdds

In [ ]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=1b1876215592662c7da22ff82654d731f0191e0dc9aba8d5028f418f65192b17
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
mkdir data

In [ ]:
rm -r data/

In [ ]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("squared")
sc = SparkContext(conf = conf)

In [ ]:
rdd1 = sc.parallelize([1,2,3,4])
print(rdd1.collect())

[1, 2, 3, 4]


We are taking this list RDD1 (1,2,3,4) and transforming it --> (1,4,9,16) by squaring each number.

 If we have a large list we can distribute it against mutliple computer nodes example Comp #1 can compute squared #1 and Comp #2 can compute squared #2 in parellel etc. etc..

**you can name rdd however you want. Example rdd can be rdd1**

* you don't care how the parallel list is distributed. You are not manually choosing how the list is distributed. The system does that

In [ ]:
rdd2 = rdd1.map(lambda x:x*x)
print(rdd2)

PythonRDD[2] at RDD at PythonRDD.scala:53


this output comes out because the calculation may be calculated at different computer nodes. Since they are computed at different nodes, if you want to know the final output you have to collect the calculations from each computer node and then it will give the correct output. instead you will use the code below.

In [ ]:
print(rdd2.collect())

[1, 4, 9, 16]


rdd is read only. So you cannot write over rdd. Therefore when you apply the x^2 function it doesn't write over rdd instead it creates a new output.

**Unlike** pandas which you can write over the dataframe and modify it.

In [ ]:
output = rdd1.map(lambda x:x*x).map(lambda x:x*2).reduce(lambda x,y:x+y)

print(output)

60


This output will generate a rdd that squares the original list and then multiplies the new rdd by 2 to generate a third rdd and then reduce it to the sum of the list.
rdd1 = [1,2,3,4]
rdd2 = [1,4,9,16] == rdd1^2
rdd3 =[2,8,18,32] == rdd2*2
Everytime a new rdd is generated in this output the previous rdd is deleted to conserve memory. So currently the only rdd stored is the one that contains the output rdd3

# **Lab 6: RDD Functions Introduction and Practice with Peter Pan Text**

In [ ]:
from pyspark import SparkConf, SparkContext
conf =SparkConf().setMaster("local").setAppName("lab6") # for Cloud and Other Servers you don' tneed to set Master environment, just use the default: setMaster()
sc = SparkContext(conf = conf)

* for Cloud and Other Servers you don' tneed to set Master environment, just use the default: setMaster()
  * .setMaste("local") will just set your environment to the local machine
* Some people choose to **.setMaster("YARN")** as the master environmnet because it will allow you to see the run time for the dataset as it provides a GUI (graphical user interface) for its users
  * This can help you to see if your code is still running.


## Practice with line filtering with lab6_test file

In [ ]:
#generate first rdd
rdd = sc.textFile("lab6_test.txt")
pythonLines =rdd.filter(lambda line: "python" in line)
#The code above generates the rdd by reading the txt file line by line and returning the line that contains the word "python"
pythonLines.count()
#there is only 1 line that contains the word "python"

1

In [ ]:
# to see the lines that were orginally in lab6_text.txt
print(rdd.collect())

['Hadoop spark hadoop', 'hadoop spark python', 'hello world']


In [ ]:
# to see the line(s) that were filtered into the rdd "PythonLines"
print(pythonLines.collect())

['hadoop spark python']


## Practice line filtering with Peter Pan file

In [ ]:
#generate peterpan rdd
rdd = sc.textFile("peterpan.txt")
peterLines =rdd.filter(lambda line: "Peter" in line)
#The code above generates the rdd by reading the txt file line by line and returning the line that contains the word "Peter"
peterLines.count()
#there are 401 linesthat contains the word "Peter"

401

In [ ]:
#generate peterpan rdd
rdd = sc.textFile("peterpan.txt")
peterLines =rdd.filter(lambda line: "Darling" in line)
#The code above generates the rdd by reading the txt file line by line and returning the line that contains the word "Darling"
peterLines.count()
#there are 114 linesthat contains the word "Darling"

114

In [ ]:
#generate peterpan rdd
rdd = sc.textFile("peterpan.txt")
peterLines =rdd.filter(lambda line: "the" in line)
#The code above generates the rdd by reading the txt file line by line and returning the line that contains the word "the"
peterLines.count()
#there are 114 linesthat contains the word "the"

2625

**So what is the rdd doing?**

The data chunks of the Peter Pan file are being stored in multiple computer nodes. Because of that the data is stored, distributedly, accross muliptle computer nodes. Then once the line that contains the word we are looking for is found, the individual computer nodes will send the line to the master node for output.

*This is the same as saying that each RDD is split into multiple partitions in Cluster.*



**If we have more computer nodes will it be faster?**

If you increase computer nodes, in theory should be faster but in actually increasing the partitions doesn't always mean that computation will be faster. It depends on the data.

In most companies 2-4 computer nodes will be enough to handle big data.
Bottleneck of the computer cluster is because the data must physically travel to each computer node via cable. The cable can take alot of time if you use 1000 nodes for a project that only needs 4 nodes.
Most companies need 4 Gigbyte or 5 Gigbyte of data. Example 6Million Yelp reviews is only 7 gigabytes. So analyzing a large portion of the PeterPan document will not take as much time as the yelp reviews because it holds much less data in terms of size.

In [ ]:
#generate peterpan rdd
rdd = sc.textFile("peterpan.txt")
peterLines =rdd.filter(lambda line: "and" in line)
#The code above generates the rdd by reading the txt file line by line and returning the line that contains the word "and"


In [ ]:
print(peterLines.collect())
#This prints the first instance of the word "and"

['This eBook is for the use of anyone anywhere at no cost and with', '       Peter Pan and Wendy', 'up, and the way Wendy knew was this. One day when she was two years old', 'she was playing in a garden, and she plucked another flower and ran with', 'Mrs. Darling put her hand to her heart and cried, “Oh, why can’t you', 'Of course they lived at 14 [their house number on their street], and', 'with a romantic mind and such a sweet mocking mouth. Her romantic', 'puzzling East, however many you discover there is always one more; and', 'though there it was, perfectly conspicuous in the right-hand corner.', 'and they all ran to her house to propose to her except Mr. Darling, who', 'took a cab and nipped in first, and so he got her. He got all of her,', 'except the innermost box and the kiss. He never knew about the box, and', 'have got it, but I can picture him trying, and then going off in a', 'and shares. Of course no one really knows, but he quite seemed to know,', 'and he often said stoc

You can run the generator(transformer) mulitple times but if you don't run the output then the cluster will not do anything. Until you tell the cluster that you want the data now. This is to save memory. So it won't actually transform the data without an output required such as **print(pythonLines.collect())**.


if you want to save the output to reuse in the future you have to use **pythonLines.persist** so that the output is stored in the memory. Otherwise the rdd won't actually save an output but rerun all generator to deliver the output everytime. This is because the output is erased everytime you run the cell for rdds.

## Basic RDD transformations of an RDD containing {1,2,3,3}

**map()**

In [ ]:
rdd = sc.parallelize([1,2,3,3])
rdd2 = rdd.map(lambda x:x*2) #This is the pyspark method of mapping transformation. This is better because you can add as many .map as you want to the transfomration
rdd2.collect()

[2, 4, 6, 6]

**filter()**

In [ ]:
rdd = sc.parallelize([1,2,3,3])
rdd2 = rdd.filter(lambda x:x!=1) #This filters out by providing values that do not equal to 1 as the rdd output.
rdd2.collect()

[2, 3, 3]

**distinct()**

remove duplicates in output

In [ ]:
rdd = sc.parallelize([1,2,3,3])
rdd2 = rdd.distinct() #This will remove the distinct features as outputs. No duplicates will be returned only the first instance.
rdd2.collect()

[1, 2, 3]

In [ ]:
rdd = sc.parallelize(["hello world","hi"])
rdd2 = rdd.map(lambda x:x.split(" ")) #this will split the words using "" as the delimeter. It will split it into multiple lists for each line/row.
rdd2.collect()

[['hello', 'world'], ['hi']]

In [ ]:
rdd = sc.parallelize(["hello world","hi"])
rdd2 = rdd.flatMap(lambda x:x.split(" ")) #this will split the words by "" and instead of generator a list of lists, it will flatten it to one list for the whole section.

['hello', 'world', 'hi']

flatMap is more readable but in reality map is faster. Either works. It just depends on how you wwant to see the output

**intersection**

In [ ]:
# you can find the union of two sets of data.
rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize([3,4,5])
rdd3 = rdd1.intersection(rdd2) #it will only output everything in both rdd1 and rdd2
print(rdd3.collect())

[3]


**subtract**

In [ ]:
# you can find the subtraction of two sets of data.
rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize([3,4,5])
rdd3 = rdd1.subtract(rdd2) #it will only output everything in rdd1-rdd2
print(rdd3.collect())

[2, 1]


**first**

In [ ]:
rdd = sc.parallelize([1,2,3,3])
rdd.first() # will output the first item in the set

1

**countByValue()**

will return a dictionary that tells you each time a number occurs in the rdd

In [ ]:
rdd = sc.parallelize([1,2,3,3])
rdd.countByValue() # will output the first item in the set

defaultdict(int, {1: 1, 2: 1, 3: 2})

## Practice Basic RDD transformations with Peter Pan txt file

**generate a list of each word using flatMap in the Peter Pan text**

In [ ]:
rdd = sc.textFile("peterpan.txt")
rdd2 = rdd.flatMap(lambda x:x.split(" ")) #to get a list of all the words in the file
rdd2.collect()

['The',
 'Project',
 'Gutenberg',
 'EBook',
 'of',
 'Peter',
 'Pan,',
 'by',
 'James',
 'M.',
 'Barrie',
 '',
 'This',
 'eBook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrictions',
 'whatsoever.',
 '',
 'You',
 'may',
 'copy',
 'it,',
 'give',
 'it',
 'away',
 'or',
 're-use',
 'it',
 'under',
 'the',
 'terms',
 'of',
 'the',
 'Project',
 'Gutenberg',
 'License',
 'included',
 'with',
 'this',
 'eBook',
 'or',
 'online',
 'at',
 'www.gutenberg.org',
 '',
 '**',
 'This',
 'is',
 'a',
 'COPYRIGHTED',
 'Project',
 'Gutenberg',
 'eBook,',
 'Details',
 'Below',
 '**',
 '**',
 '',
 '',
 '',
 '',
 'Please',
 'follow',
 'the',
 'copyright',
 'guidelines',
 'in',
 'this',
 'file.',
 '',
 '',
 '',
 '',
 '**',
 '',
 'Title:',
 'Peter',
 'Pan',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Peter',
 'Pan',
 'and',
 'Wendy',
 '',
 'Author:',
 'James',
 'M.',
 'Barrie',
 '',
 'Posting',
 'Date:',
 'June',
 '25,',
 '2008',
 '[

**generate a dictionary of the count of each word using countByValue in the Peter Pan text**

In [ ]:
rdd = sc.textFile("peterpan.txt")
rdd2 = rdd.flatMap(lambda x:x.split(" ")) #to get a list of all the words in the file
rdd2.countByValue() # to get the count of each word in the form of a dictionary


defaultdict(int,
            {'The': 151,
             'Project': 79,
             'Gutenberg': 22,
             'EBook': 2,
             'of': 929,
             'Peter': 238,
             'Pan,': 4,
             'by': 175,
             'James': 8,
             'M.': 4,
             'Barrie': 4,
             '': 2259,
             'This': 32,
             'eBook': 9,
             'is': 350,
             'for': 377,
             'the': 2331,
             'use': 19,
             'anyone': 4,
             'anywhere': 2,
             'at': 322,
             'no': 136,
             'cost': 2,
             'and': 1396,
             'with': 361,
             'almost': 36,
             'restrictions': 2,
             'whatsoever.': 2,
             'You': 36,
             'may': 49,
             'copy': 8,
             'it,': 63,
             'give': 26,
             'it': 473,
             'away': 29,
             'or': 133,
             're-use': 2,
             'under': 28,
             'ter

**To find the frequency of a certain word in Peter Pan text**
According to the code below. The word "the" appears 151 times in the text.


In [ ]:
rdd = sc.textFile("peterpan.txt")
rdd2 = rdd.flatMap(lambda x:x.split(" ")) #to get a list of all the words in the file
output=rdd2.countByValue() # to get the count of each word in the form of a dictionary and save to output
print(output)
print(output["the"])


defaultdict(<class 'int'>, {'The': 151, 'Project': 79, 'Gutenberg': 22, 'EBook': 2, 'of': 929, 'Peter': 238, 'Pan,': 4, 'by': 175, 'James': 8, 'M.': 4, 'Barrie': 4, '': 2259, 'This': 32, 'eBook': 9, 'is': 350, 'for': 377, 'the': 2331, 'use': 19, 'anyone': 4, 'anywhere': 2, 'at': 322, 'no': 136, 'cost': 2, 'and': 1396, 'with': 361, 'almost': 36, 'restrictions': 2, 'whatsoever.': 2, 'You': 36, 'may': 49, 'copy': 8, 'it,': 63, 'give': 26, 'it': 473, 'away': 29, 'or': 133, 're-use': 2, 'under': 28, 'terms': 21, 'License': 8, 'included': 3, 'this': 195, 'online': 4, 'www.gutenberg.org': 2, '**': 4, 'a': 962, 'COPYRIGHTED': 1, 'eBook,': 2, 'Details': 1, 'Below': 1, 'Please': 2, 'follow': 8, 'copyright': 16, 'guidelines': 1, 'in': 683, 'file.': 1, 'Title:': 1, 'Pan': 13, 'Wendy': 200, 'Author:': 1, 'Posting': 1, 'Date:': 2, 'June': 1, '25,': 1, '2008': 1, '[EBook': 1, '#16]': 1, 'Release': 1, 'July,': 1, '1991': 1, 'Last': 2, 'Updated:': 1, 'October': 1, '14,': 1, '2016': 1, 'Language:': 1, '

**how do we get rid of punctuations?**

previously we've been using **lambda** but that is not the best method instead you can define a function.

In [ ]:
def cleanTextData(eachrow):
  lowercase =eachrow.lower() #first lower all words so that you get the correct word count
  #you can add as many nlp cleansing lines as you possibly want.
  return lowercase.split(" ")

rdd= sc.textFile("peterpan.txt")
rdd2= rdd.flatMap(cleanTextData)
output = rdd2.countByValue()
print(output)
print(output["the"])

defaultdict(<class 'int'>, {'the': 2511, 'project': 83, 'gutenberg': 25, 'ebook': 13, 'of': 976, 'peter': 243, 'pan,': 4, 'by': 187, 'james': 8, 'm.': 4, 'barrie': 4, '': 2259, 'this': 234, 'is': 351, 'for': 408, 'use': 22, 'anyone': 4, 'anywhere': 2, 'at': 340, 'no': 150, 'cost': 2, 'and': 1425, 'with': 373, 'almost': 37, 'restrictions': 2, 'whatsoever.': 2, 'you': 447, 'may': 49, 'copy': 8, 'it,': 63, 'give': 27, 'it': 594, 'away': 30, 'or': 147, 're-use': 2, 'under': 31, 'terms': 23, 'license': 12, 'included': 3, 'online': 4, 'www.gutenberg.org': 2, '**': 4, 'a': 981, 'copyrighted': 2, 'ebook,': 2, 'details': 1, 'below': 4, 'please': 5, 'follow': 9, 'copyright': 17, 'guidelines': 1, 'in': 719, 'file.': 1, 'title:': 1, 'pan': 16, 'wendy': 202, 'author:': 1, 'posting': 1, 'date:': 2, 'june': 1, '25,': 1, '2008': 1, '[ebook': 1, '#16]': 1, 'release': 3, 'july,': 1, '1991': 1, 'last': 51, 'updated:': 1, 'october': 1, '14,': 1, '2016': 1, 'language:': 1, 'english': 4, 'character': 3, 'se

defining function is better for readability and being able to edit/adapt functions more easily before passing it to the spark operators.

##Saving RDD Outputs
outputs should not be saved as collect because it is too memory intensive for large datasets on a single machine. **INSTEAD** you should write/save the output to an output folder/file

In [ ]:
rdd = sc.textFile("peterpan.txt")
rdd2 = rdd.flatMap(lambda x:x.split(" ")) #to get a list of all the words in the file
rdd2.saveAsTextFile("output")
